In [103]:
import pickle
import requests
from datetime import date,timedelta
from io import StringIO
import pandas as pd
import json
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
import os, json
from pandas import Timestamp

In [107]:
def find(arr , name):
    for idx,item in enumerate(arr):
        #print(x['name'])
        if item["name"] == name:
            return idx
    return False


result=[]
for root, dirs, files in os.walk("./stock_all_backup/"):
    for file in files:
        if file.endswith(".json"):
#             print(os.path.join(root, file))
            dirname = root.split('/')[-1]
            date = file.split('_')[0]
            stock_id = file.split('_')[1].split('.')[0]
            if(find(result , stock_id)):
                index = find(result , stock_id)
                result[index]['dates'].append({'date':date, 'file':dirname+'/'+file})
            else:
                result.append({
                    'name':stock_id,
                    'dates':[{
                        'date':date,
                        'file':dirname+'/'+file
                    }]
                })
                

json_object = json.dumps(result, indent=2)
with open('data_list_2022.json', 'w') as json_file:
    json_file.write(json_object)
# result[0]['areas'].append({'ddd':'ddds'})
# result

Index(['1101', '1102', '1103', '1104', '1108', '1109', '1110', '1201', '1203',
       '1210',
       ...
       '8936', '8937', '8938', '8941', '8942', '9949', '9950', '9951', '9960',
       '9962'],
      dtype='object', name='stock_id', length=1774)

In [27]:
filepath = './json/data_1102.json'
if os.path.exists(filepath):
    print('true')

true


In [11]:
high=high.fillna(0)
low=low.fillna(0)
start=start.fillna(0)
end=end.fillna(0)
volumn=volumn.fillna(0)
three=three.fillna(0)
margin=margin.fillna(0)

In [69]:
margin.index

MultiIndex([('1101', '2003-02-12'),
            ('1101', '2003-02-13'),
            ('1101', '2003-02-14'),
            ('1101', '2003-02-17'),
            ('1101', '2003-02-18'),
            ('1101', '2003-02-19'),
            ('1101', '2003-02-20'),
            ('1101', '2003-02-21'),
            ('1101', '2003-02-24'),
            ('1101', '2003-02-25'),
            ...
            ('9962', '2022-10-18'),
            ('9962', '2022-10-19'),
            ('9962', '2022-10-20'),
            ('9962', '2022-10-21'),
            ('9962', '2022-10-24'),
            ('9962', '2022-10-25'),
            ('9962', '2022-10-26'),
            ('9962', '2022-10-27'),
            ('9962', '2022-10-28'),
            ('9962', '2022-10-31')],
           names=['stock_id', 'date'], length=5901028)

In [12]:
('2330','2022-11-03') in three.index

True

In [60]:
margin.loc['1459']#.loc['2022-']['融券差額(張)'],

,融資(張),融資差額(張),融資使用率,融券(張),融券差額(張),券資比
date,,,,,,
2003-02-12,24401.0,-807.0,28.23,836.0,183.0,3.43
2003-02-13,21997.0,-2404.0,25.45,643.0,-193.0,2.92
2003-02-14,22366.0,369.0,25.87,532.0,-111.0,2.38
2003-02-17,22040.0,-326.0,25.50,542.0,10.0,2.46
2003-02-18,21653.0,-387.0,25.05,451.0,-91.0,2.08
...,...,...,...,...,...,...
2022-10-25,801.0,0.0,0.00,0.0,0.0,0.00
2022-10-26,799.0,-2.0,0.00,0.0,0.0,0.00
2022-10-27,799.0,0.0,0.00,0.0,0.0,0.00


In [49]:
# start['1101']['2022-10-31']
today = datetime.today()
for i in tqdm(start.columns):
    result=[]
    date=[]
    startday_str = '01/4/21 8:00:00'
    startday = datetime.strptime(startday_str, '%m/%d/%y %H:%M:%S')
    startday_ymd = startday.strftime("%Y-%m-%d")
    
    df0 = pd.DataFrame(end[i])
    df0.columns=['收盤價']
    df0['ma5']=round(df0["收盤價"].rolling(window=5).mean(),2)
    df0['ma10']=round(df0["收盤價"].rolling(window=10).mean(),2)
    df0['ma20']=round(df0["收盤價"].rolling(window=20).mean(),2)
    df0['ma60']=round(df0["收盤價"].rolling(window=60).mean(),2)
    
    filepath = './json/data_'+i+'.json'
    if os.path.exists(filepath):
        with open(filepath, mode='r+') as f:
            data = json.load(f)
            lastupdate_str = data['date'][-1]+ ' 8:00:00'
            lastupdate = datetime.strptime(lastupdate_str, '%Y-%m-%d %H:%M:%S')
            lastupdate_ymd = data['date'][-1]

            while lastupdate < today:
                try:
                    timestr = float(lastupdate.timestamp())*1000
                    lastupdate_ymd = lastupdate.strftime("%Y-%m-%d")

                    if((i,lastupdate_ymd) in margin.index):
                        mar=margin.loc[i].loc[lastupdate_ymd]['融券差額(張)']
                    else:
                        mar=0

                    if((i,lastupdate_ymd) in three.index):
                        thr=three.loc[i].loc[lastupdate_ymd]['投信買賣超(張)']
                    else:
                        thr=0

                    data['stock_data'].append(
                        [timestr,start[i][lastupdate_ymd],high[i][lastupdate_ymd],low[i][lastupdate_ymd],end[i][lastupdate_ymd],
                         round(volumn[i][lastupdate_ymd]/1000,2),thr,mar,
                        df0.loc[lastupdate_ymd]['ma5'],df0.loc[lastupdate_ymd]['ma10'],df0.loc[lastupdate_ymd]['ma20'],df0.loc[lastupdate_ymd]['ma60']
                        ])
                    data['date'].append(lastupdate_ymd)
                        #date.append(lastupdate_ymd)
                except:
                    pass
                lastupdate = lastupdate + timedelta(days=1)
                #print(data['date'])

            f.seek(0) 
            json.dump(data, f,indent=2)
            f.truncate() 
    else:
        print(i)
        while startday < today:
            try:
                timestr = float(startday.timestamp())*1000
                startday_ymd = startday.strftime("%Y-%m-%d")

                if((i,startday_ymd) in margin.index):
                    mar=margin.loc[i].loc[startday_ymd]['融券差額(張)']
                else:
                    mar=0

                if((i,startday_ymd) in three.index):
                    thr=three.loc[i].loc[startday_ymd]['投信買賣超(張)']
                else:
                    thr=0
                
                result.append(
                    [timestr,start[i][startday_ymd],high[i][startday_ymd],low[i][startday_ymd],end[i][startday_ymd],
                     round(volumn[i][startday_ymd]/1000,2),thr,mar,
                    df0.loc[startday_ymd]['ma5'],df0.loc[startday_ymd]['ma10'],df0.loc[startday_ymd]['ma20'],df0.loc[startday_ymd]['ma60']
                    ])
                date.append(startday_ymd)
            except:
                pass


            startday = startday + timedelta(days=1)
        dictionary = {
                        "stock_id": i,
                        "date": date,
                        "stock_data": result,
                    }


        json_object = json.dumps(dictionary, indent=2)
        with open('./json/data_'+str(i)+'.json', 'w') as json_file:
            json_file.write(json_object)

4951
6695



In [47]:
today = datetime.today()

filepath = './json/data_2330.json'
i='2330'
df0 = pd.DataFrame(end[i])
df0.columns=['收盤價']
df0['ma5']=round(df0["收盤價"].rolling(window=5).mean(),2)
df0['ma10']=round(df0["收盤價"].rolling(window=10).mean(),2)
df0['ma20']=round(df0["收盤價"].rolling(window=20).mean(),2)
df0['ma60']=round(df0["收盤價"].rolling(window=60).mean(),2)
if os.path.exists(filepath):
    with open(filepath, mode='r+') as f:
        data = json.load(f)
        lastupdate_str = data['date'][-1]+ ' 8:00:00'
        lastupdate = datetime.strptime(lastupdate_str, '%Y-%m-%d %H:%M:%S')
        lastupdate_ymd = data['date'][-1]

        while lastupdate < today:
            try:
                timestr = float(lastupdate.timestamp())*1000
                lastupdate_ymd = lastupdate.strftime("%Y-%m-%d")

                if((i,lastupdate_ymd) in margin.index):
                    mar=margin.loc[i].loc[lastupdate_ymd]['融券差額(張)']
                else:
                    mar=0

                if((i,lastupdate_ymd) in three.index):
                    thr=three.loc[i].loc[lastupdate_ymd]['投信買賣超(張)']
                else:
                    thr=0
                
                data['stock_data'].append(
                    [timestr,start[i][lastupdate_ymd],high[i][lastupdate_ymd],low[i][lastupdate_ymd],end[i][lastupdate_ymd],
                     round(volumn[i][lastupdate_ymd]/1000,2),thr,mar,
                    df0.loc[lastupdate_ymd]['ma5'],df0.loc[lastupdate_ymd]['ma10'],df0.loc[lastupdate_ymd]['ma20'],df0.loc[lastupdate_ymd]['ma60']
                    ])
                data['date'].append(lastupdate_ymd)
                    #date.append(lastupdate_ymd)
            except:
                pass
            lastupdate = lastupdate + timedelta(days=1)
            #print(data['date'])

        f.seek(0) 
        json.dump(data, f,indent=2)
        f.truncate() 

In [50]:
with os.scandir('./json/') as it:
    for entry in tqdm(it):
        if entry.name.endswith(".json") and entry.is_file():
            with open(entry.path) as f:
                data = json.load(f)
                #print(data['stock_data'])
                with open('./json_part/'+entry.name, 'w') as json_file:
                    json_file.write(str(data['stock_data']))